# Entrenamiento y evaluación de modelos

En esta sección se entrenan y evalúan tres modelos distintos:
- k-Nearest Neighbors (kNN)
- Modelo de ensamble (Random Forest)
- Deep Neural Network (DNN, mínimo 3 capas ocultas)

Se muestra una tabla comparativa de desempeño en X_train, X_val y X_test.

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from scikeras.wrappers import KerasRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import pandas as pd

# Función para evaluar modelos
def eval_model(model, X_train, X_val, X_test, y_train, y_val, y_test):
    y_pred_train = model.predict(X_train)
    y_pred_val = model.predict(X_val)
    y_pred_test = model.predict(X_test)
    return {
        'train_rmse': np.sqrt(mean_squared_error(y_train, y_pred_train)),
        'val_rmse': np.sqrt(mean_squared_error(y_val, y_pred_val)),
        'test_rmse': np.sqrt(mean_squared_error(y_test, y_pred_test)),
        'train_r2': r2_score(y_train, y_pred_train),
        'val_r2': r2_score(y_val, y_pred_val),
        'test_r2': r2_score(y_test, y_pred_test)
    }

# Preprocesar datos
X_train_proc = pipeline.fit_transform(X_train)
X_val_proc = pipeline.transform(X_val)
X_test_proc = pipeline.transform(X_test)

# 1. k-Nearest Neighbors
knn = KNeighborsRegressor(n_neighbors=5)
knn.fit(X_train_proc, y_train)
knn_results = eval_model(knn, X_train_proc, X_val_proc, X_test_proc, y_train, y_val, y_test)

# 2. Random Forest
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train_proc, y_train)
rf_results = eval_model(rf, X_train_proc, X_val_proc, X_test_proc, y_train, y_val, y_test)

# 3. Deep Neural Network
def build_dnn():
    model = Sequential([
        Dense(128, activation='relu', input_shape=(X_train_proc.shape[1],)),
        Dropout(0.2),
        Dense(64, activation='relu'),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

dnn = KerasRegressor(model=build_dnn, epochs=50, batch_size=32, verbose=0)
dnn.fit(X_train_proc, y_train)
dnn_results = eval_model(dnn, X_train_proc, X_val_proc, X_test_proc, y_train, y_val, y_test)

# Tabla comparativa
results = pd.DataFrame([
    {'Modelo': 'kNN', **knn_results},
    {'Modelo': 'Random Forest', **rf_results},
    {'Modelo': 'DNN', **dnn_results}
])
print("Resultados de evaluación:")
results[['Modelo', 'train_rmse', 'val_rmse', 'test_rmse', 'train_r2', 'val_r2', 'test_r2']]

KeyboardInterrupt: 

# Procesamiento y limpieza de datos

En esta sección se aplican buenas prácticas de procesamiento y limpieza de datos antes de entrenar los modelos:
- Manejo de valores nulos
- Codificación de variables categóricas
- Normalización/estandarización
- Reducción de dimensionalidad (si aplica)
- Pipeline de procesamiento con scikit-learn
- División de datos en X_train, X_val y X_test (70/15/15)

**Ejecuta todas las celdas de esta sección antes de pasar a la sección de modelos.**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA

# Cargar datos
df = pd.read_csv('CarPrice_Assignment.csv')
df_original = df.copy()  # Para referencia

df.info()
df.head()

In [ ]:
# 1. Manejo de valores nulos
df = df.dropna(axis=1, how='all')  # Eliminar columnas completamente nulas
for col in df.columns:
    if df[col].isnull().sum() > 0:
        print(f"Columna '{col}' tiene {df[col].isnull().sum()} valores nulos")
# Imputación se hará en el pipeline

In [ ]:
# 2. Identificación de variables categóricas y numéricas
cat_cols = df.select_dtypes(include=['object']).columns.tolist()
num_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Remover la variable objetivo de las variables numéricas si está presente
target = 'price'
if target in num_cols:
    num_cols.remove(target)

print('Variables categóricas:', cat_cols)
print('Variables numéricas:', num_cols)

In [ ]:
# 3. Definir el pipeline de procesamiento
target = 'price'  # Cambia si tu variable objetivo tiene otro nombre
X = df.drop(columns=[target])
y = df[target]

# Preprocesamiento para variables numéricas y categóricas
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])
cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer([
    ('num', num_pipeline, num_cols),
    ('cat', cat_pipeline, cat_cols)
])

# Reducción de dimensionalidad (opcional, solo si hay muchas variables)
use_pca = len(num_cols) + len(cat_cols) > 15
if use_pca:
    pipeline = Pipeline([
        ('pre', preprocessor),
        ('pca', PCA(n_components=0.95))
    ])
else:
    pipeline = Pipeline([
        ('pre', preprocessor)
    ])

In [ ]:
# 4. División de datos en train, val y test (70/15/15)
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.1765, random_state=42)  # 0.1765*0.85 ≈ 0.15
print(f"Tamaños: Train={X_train.shape}, Val={X_val.shape}, Test={X_test.shape}")